# TODO
- TF/IDF
- get ratings
- partial tags match (female harem matches harem)
- get more data
- iteractive fine tuning of the search (showing candiate tags)
- store and remember (profile, liked animes in the past)
- scrape 'sources' -> ratings
- ai turn (scraped) summary to tags
- age
- additional criteria (episodes, seasons, etc)

In [61]:
import pandas as pd 
import json

In [62]:
with open("./anime-offline-database.json", "r") as file:
    jsondata = json.load(file)

In [63]:
items = jsondata['data']

In [64]:
for item in items:
    if "One Piece" != item['title']:
        continue
        
    print(item)


{'sources': ['https://anidb.net/anime/69', 'https://anilist.co/anime/21', 'https://anime-planet.com/anime/one-piece', 'https://anisearch.com/anime/2227', 'https://kitsu.app/anime/12', 'https://livechart.me/anime/321', 'https://myanimelist.net/anime/21', 'https://notify.moe/anime/jdZp5KmiR'], 'title': 'One Piece', 'type': 'TV', 'episodes': 1122, 'status': 'ONGOING', 'animeSeason': {'season': 'FALL', 'year': 1999}, 'picture': 'https://cdn.myanimelist.net/images/anime/1244/138851.jpg', 'thumbnail': 'https://cdn.myanimelist.net/images/anime/1244/138851t.jpg', 'duration': {'value': 1440, 'unit': 'SECONDS'}, 'synonyms': ['1P', "All'arrembaggio!", 'All`arrembaggio!', 'Budak Getah', 'Lastik Çocuk', 'ONE PIECE', 'OP', 'OP TV', 'One Piece TV', 'OneP', 'Tek parça', "Tutti all'arrembaggio!", 'Vua Hải Tặc', 'optv', 'Đảo hải tặc', 'Ντρέηκ, το Κυνήγι του Θησαυρού', 'Ντρέικ και το Κυνήγι του Θησαυρού', 'Большой куш', 'Ван-Пис', 'Великий куш', 'Едно Цяло', 'Одним куском', 'Уан Пийс', 'וואן פיס', 'وان پ

In [65]:
rows = []
for item in items:
    tup = (
    item['title'],
    item['type'],
    item['tags'],
    item['episodes']
        )
    rows.append(tup)

In [66]:
print(len(rows))

34955


In [67]:
df = pd.DataFrame(rows, columns=["title", "type", "tags", "episodes"])

In [68]:
df

,title,type,tags,episodes
0,!NVADE SHOW!,SPECIAL,"[band, full cgi, music, primarily female cast,...",1
1,"""0""",SPECIAL,"[drama, female protagonist, indefinite, music,...",1
2,"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokub...",MOVIE,"[family friendly, fantasy, frogs, kids]",1
3,"""Ai"" wo Taberu",MOVIE,[drama],1
4,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",OVA,"[asia, based on a light novel, comedy, drama, ...",1
...,...,...,...,...
34950,▲MEW▲△MEW△CAKE,SPECIAL,[],1
34951,◯,ONA,"[abstract, angst, avant garde, black and white...",1
34952,「S.K.T(Swallow Knights Tales)」 Weptun Teureill...,SPECIAL,"[action, comedy, fantasy]",1
34953,空の境界,TV,"[action, mystery, romance, supernatural, suspe...",13


In [69]:
has_tags = 0
tv_shows = 0
all_tags = []
for idx, row in df.iterrows():
    if row['type'] != "TV":
        continue
    tv_shows += 1
    tags = row['tags']
    if len(tags) > 0:
        has_tags += 1

        for tag in tags:
            if not tag in all_tags:
                all_tags.append(tag)
        
print("tv shows", tv_shows)
print("rows with tags", has_tags)
print("tags", len(all_tags))

tv shows 8880
rows with tags 8818
tags 1147


In [70]:
print(all_tags[:20])

['acting', 'age regression', 'anti-hero', 'award winning', 'based on a manga', 'bowling', 'bullying', 'coming of age', 'daily life', 'dancing', 'denpa', 'detective', 'drama', 'ensemble cast', 'fake relationship', 'family life', 'female harem', 'female protagonist', 'filmmaking', 'genius']


In [71]:
df['episodes'].mean()

np.float64(13.0160205979116)

In [72]:
df['episodes'].max()

np.int64(3057)

# find common tags

In [73]:
tags_to_match = ['adventure', 'fantasy', 'male', 'romance']

In [74]:
def find_matches(search_tags,anime_tags):
    matches = 0
    for tag in search_tags:
        if tag in anime_tags:
            matches += 1
    match_ratio = matches*100/len(anime_tags)
    return matches, match_ratio

In [75]:
matching_animes = []
match_count = {
    0: 0, 
    1: 0,
    2: 0,
    3: 0,
    4: 0,
}

for idx, row in df.iterrows():
    if row['type'] != "TV":
        continue
    tags = row['tags']
    if len(tags) == 0:
        continue
    matches, match_ratio = find_matches(tags_to_match, tags)
    if matches == 0:
        continue
        
    match_count[matches] += 1
    anime = (row['title'], matches, match_ratio)
    matching_animes.append(anime)
print(match_count)
print(len(matching_animes))

{0: 0, 1: 3187, 2: 1666, 3: 368, 4: 0}
5221


In [76]:
matching_df = pd.DataFrame(matching_animes, columns=['title', 'matches', 'ratio'])

In [77]:
matching_df.sort_values(["matches", "ratio"], ascending=[False, False])

,title,matches,ratio
3717,Saru Getchu: On Air 2nd,3,42.857143
4705,Wagamama☆Fairy Mirumo de Pon! Wonderful,3,42.857143
4749,Willy Fog 2: Part 1,3,37.500000
5112,"Yuusha Party wo Tsuihou sareta Shiromadoushi, ...",3,37.500000
4704,Wagamama☆Fairy Mirumo de Pon! Golden,3,30.000000
...,...,...,...
2231,Kiseijuu: Sei no Kakuritsu,1,1.515152
2182,Kill la Kill,1,1.388889
1078,Elfen Lied,1,1.351351
1093,Ergo Proxy,1,1.351351
